In [1]:
import numpy as np
from scipy.linalg import lu
from scipy.sparse.linalg import svds
from numpy.linalg import svd

In [2]:
def check_spanrd(vectors, d):
    """
    Inputs:
        - vectors (array): matrix (N, d)
        - d (int): dimension of the space to be spanned
    Return:
        - True or False
    """
    # https://math.stackexchange.com/questions/56201/how-to-tell-if-a-set-of-vectors-spans-a-space
    # https://stackoverflow.com/questions/15638650/is-there-a-standard-solution-for-gauss-elimination-in-python
    pl, u = lu(vectors, permute_l=True)
    rank = np.linalg.matrix_rank(u)
    return d == int(rank)

def span(vectors):
    
    d = vectors.shape[1]
    for i in range(d):
        if check_spanrd(vectors, d - i):
            return d - i

In [3]:
files = ["lastfm_d6_span2.npz"]

dims = [8, 9, 10, 13, 14, 15, 20, 21, 24, 25, 28, 29, 37, 38, 39, 40, 43, 44, 46, 47]
spns = [6, 8, 10, 13, 14, 15, 20, 19, 21, 24, 27, 27, 37, 35, 39, 40, 31, 44, 43, 43]

files = ["lastfm_d{0}_span{1}.npz".format(i,j) for i,j in zip(dims,spns)]

features = {}
thetas = {}

for file, d in zip(files, dims):
    
        f = np.load(file)
        features[d] = f['features']
        thetas[d] = f['theta']
        print("Loaded d={}".format(d))
        del(f)
print()

Loaded d=8
Loaded d=9
Loaded d=10
Loaded d=13
Loaded d=14
Loaded d=15
Loaded d=20
Loaded d=21
Loaded d=24
Loaded d=25
Loaded d=28
Loaded d=29
Loaded d=37
Loaded d=38
Loaded d=39
Loaded d=40
Loaded d=43
Loaded d=44
Loaded d=46
Loaded d=47



In [4]:
# remove useless features

tol = 1e-8  # threshold to consider an eigenvalue equal to zero

new_features = {}
new_thetas = {}

for d in dims:
    
    print("Starting d={}".format(d))
    fmat = features[d].reshape(-1, d)
    
    U, s, Vt = svd(fmat, full_matrices=False)
    sp = np.sum(s > tol)
    print("[d={0}] span: {1}".format(d,sp))
    s = s[:sp]
    U = U[:, :sp]
    Vt = Vt[:sp, :]

    s = np.diag(s)
    U = np.dot(U, s)
    M = U.dot(Vt)
    rmse = np.sqrt(np.mean(np.abs(M - fmat) ** 2))
    print("[d={0}] Reconstruction rmse: {1}".format(d, rmse))
    
    idx = (d, sp)
        
    # create new features/parameters
    new_features[idx] = U.reshape(features[d].shape[0], features[d].shape[1], sp)
    new_thetas[idx] = Vt.dot(thetas[d])
    
    # normalize parameters
    norm = np.linalg.norm(new_thetas[idx])
    new_thetas[idx] /= norm
    new_features[idx] *= norm
    
    # check errors
    old_mu = features[d].dot(thetas[d])
    new_mu = new_features[idx].dot(new_thetas[idx])
    err = np.abs(old_mu - new_mu)
    print("[d={0}] mu error: max {1} - mean {2}".format(d, np.max(err), np.mean(err)))
    
    del(old_mu)
    del(new_mu)
    del(err)
    
    print()

Starting d=8
[d=8] span: 7
[d=8] Reconstruction rmse: 7.8126120683919e-08
[d=8] mu error: max 9.5367431640625e-07 - mean 7.775871324611217e-08

Starting d=9
[d=9] span: 9
[d=9] Reconstruction rmse: 1.3467854387272382e-07
[d=9] mu error: max 2.6226043701171875e-06 - mean 3.920620983421941e-08

Starting d=10
[d=10] span: 10
[d=10] Reconstruction rmse: 7.502294607775184e-08
[d=10] mu error: max 2.86102294921875e-06 - mean 4.9370900256917594e-08

Starting d=13
[d=13] span: 13
[d=13] Reconstruction rmse: 6.96845887659947e-08
[d=13] mu error: max 2.384185791015625e-06 - mean 4.0059727979269155e-08

Starting d=14
[d=14] span: 14
[d=14] Reconstruction rmse: 9.085532326480461e-08
[d=14] mu error: max 1.9073486328125e-06 - mean 3.258908876091482e-08

Starting d=15
[d=15] span: 15
[d=15] Reconstruction rmse: 7.09447434132926e-08
[d=15] mu error: max 3.337860107421875e-06 - mean 2.966848455798754e-08

Starting d=20
[d=20] span: 20
[d=20] Reconstruction rmse: 3.4120427017114707e-07
[d=20] mu error:

In [5]:
# check prediction errors and select ground-truth representation

# load data

data_path = "lastfmlog.npy"
ratings = np.load(data_path)
ratings = (ratings - np.mean(ratings)) / np.std(ratings)

d_gt = None
min_mse = 1

for d in new_features.keys():
    mu = new_features[d].dot(new_thetas[d])
    mse = np.mean(np.abs(mu - ratings)**2)
    print("{0} MSE: {1}".format(d, mse))
    if mse < min_mse:
        d_gt = d
        min_mse = mse
    del(mu)

print()
print("Ground truth: {0} - MSE: {1}".format(d_gt, min_mse))

d_gt = (10, 10)

(46, 43) MSE: 0.11520074247623423
(47, 44) MSE: 0.07854247685722908
(21, 20) MSE: 0.06423985070818058
(28, 28) MSE: 0.09398516381272812
(39, 39) MSE: 0.12367216370428877
(13, 13) MSE: 0.1294628902212501
(24, 23) MSE: 0.11597057996725627
(43, 31) MSE: 0.0709666678821771
(25, 24) MSE: 0.09690522274404312
(20, 20) MSE: 0.111237963661736
(44, 44) MSE: 0.13290080582877026
(9, 9) MSE: 0.12558174867756924
(29, 28) MSE: 0.061676938454786695
(10, 10) MSE: 0.06375650858896147
(40, 40) MSE: 0.16757244154473758
(8, 7) MSE: 0.06047927825471387
(14, 14) MSE: 0.12197448149594643
(15, 15) MSE: 0.13113767183087255
(38, 37) MSE: 0.0743236263866338
(37, 37) MSE: 0.22995321687881973

Ground truth: (8, 7) - MSE: 0.06047927825471387


In [6]:
# filter gaps

thresh = 0.01

# ground truth
mu_gt = new_features[d_gt].dot(new_thetas[d_gt])
gap_gt = np.max(mu_gt, axis=1)[:, np.newaxis] - mu_gt
gap_gt[gap_gt == 0] = 100
print("gap min:", gap_gt.min())
gap_gt = np.min(gap_gt, axis=1)

# indexes of contexts with minimum gap above threshold
good_contexts = gap_gt > thresh
print("# contexts with gap_min > {0}: {1}".format(thresh, np.sum(good_contexts)))

# filter
for d in new_features.keys():
    new_features[d] = new_features[d][good_contexts, :, :]

n_contexts = np.sum(good_contexts)
mu_gt = mu_gt[good_contexts, :]

gap min: 0.00025200844
# contexts with gap_min > 0.01: 1271


In [7]:
# check misspecification

for d in new_features.keys():
    mu = new_features[d].dot(new_thetas[d])
    err = np.abs(mu - mu_gt)
    print("[d={0}] error wrt ground truth: max {1} - min {2} - mean {3}".format(d, err.max(), np.min(err), np.mean(err)))
    err_cont = np.min(err, axis=1)
    print("[d={0}] min error per context: max {1} - min {2} - mean {3}".format(d, err_cont.max(), np.min(err_cont), np.mean(err_cont)))
    del(mu)
    del(err)

[d=(46, 43)] error wrt ground truth: max 5.5605058670043945 - min 0.0 - mean 0.0915919616818428
[d=(46, 43)] min error per context: max 0.0008884072303771973 - min 0.0 - mean 4.998865915695205e-05
[d=(47, 44)] error wrt ground truth: max 4.564278602600098 - min 5.960464477539063e-08 - mean 0.08259787410497665
[d=(47, 44)] min error per context: max 0.004176050424575806 - min 5.960464477539063e-08 - mean 0.00022634323977399617
[d=(21, 20)] error wrt ground truth: max 5.423555850982666 - min 1.7881393432617188e-06 - mean 0.08175911009311676
[d=(21, 20)] min error per context: max 0.0035548508167266846 - min 1.7881393432617188e-06 - mean 0.002971914131194353
[d=(28, 28)] error wrt ground truth: max 6.105203151702881 - min 7.152557373046875e-06 - mean 0.0858202874660492
[d=(28, 28)] min error per context: max 0.0002554357051849365 - min 7.152557373046875e-06 - mean 0.0002516135573387146
[d=(39, 39)] error wrt ground truth: max 5.763015270233154 - min 3.933906555175781e-06 - mean 0.10508774

In [8]:
# check span optimal arms

span_opt = {}

for d in new_features.keys():
    
    mu = new_features[d].dot(new_thetas[d])
    astar = np.argmax(mu, axis=1)
    fstar = np.array([new_features[d][x, astar[x]] for x in range(n_contexts)])

    span = d[1]
    for i in range(d[1]):
        if check_spanrd(fstar, d[1] - i):
            span = d[1] - i
            break
    
    span_opt[d] = span
    
    outer = np.matmul(fstar.T, fstar) / n_contexts
    lambda_hls = np.linalg.eigvals(outer).min()
    
    print("[d={0}] span optimal arms: {1} - lambda HLS: {2}".format(d, span, lambda_hls))
    
    del(mu)
    del(astar)
    del(fstar)
    del(outer)

[d=(46, 43)] span optimal arms: 43 - lambda HLS: 0.007056571077555418
[d=(47, 44)] span optimal arms: 43 - lambda HLS: -7.231829131407695e-16
[d=(21, 20)] span optimal arms: 19 - lambda HLS: -2.327827486325873e-09
[d=(28, 28)] span optimal arms: 27 - lambda HLS: 1.2351413225530905e-09
[d=(39, 39)] span optimal arms: 39 - lambda HLS: 0.10092531889677048
[d=(13, 13)] span optimal arms: 13 - lambda HLS: 0.08324939012527466
[d=(24, 23)] span optimal arms: 21 - lambda HLS: -1.0010548066929914e-05
[d=(43, 31)] span optimal arms: 31 - lambda HLS: 0.00019171684107277542
[d=(25, 24)] span optimal arms: 24 - lambda HLS: 0.03321532905101776
[d=(20, 20)] span optimal arms: 20 - lambda HLS: 0.006384613923728466
[d=(44, 44)] span optimal arms: 44 - lambda HLS: 0.12242113798856735
[d=(9, 9)] span optimal arms: 8 - lambda HLS: -2.154773334918421e-16
[d=(29, 28)] span optimal arms: 27 - lambda HLS: 3.19593039099092e-13
[d=(10, 10)] span optimal arms: 10 - lambda HLS: 0.030743177980184555
[d=(40, 40)] s

In [26]:
# save

for i, d in enumerate(new_features.keys()):
    np.savez_compressed('lastfm_post_{0}_d{1}_span{2}{3}.npz'.format(i, d[1],span_opt[d], "_gt" if d == d_gt else ""), 
                        features=new_features[d], theta=new_thetas[d])